In [1]:
import torch # type: ignore[attr-defined]
from transformers import WhisperForConditionalGeneration, WhisperProcessor # type: ignore[attr-defined]
from causal_wrapper import load_causal_whisper
from utils import prepare_data

/home/vansh/whisper-stream/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'WhisperEncoder' is not defined

In [ ]:
MODEL_ID = "openai/whisper-base"
DEVICE = "cuda:6" if torch.cuda.is_available() else "cpu"

In [ ]:
my_model = load_causal_whisper(MODEL_ID, device=DEVICE)
whisper_model = WhisperForConditionalGeneration.from_pretrained(MODEL_ID)
whisper_model.to(DEVICE)
processor = WhisperProcessor.from_pretrained(MODEL_ID)    

In [ ]:
ds = prepare_data(max_shards=1)

In [ ]:
my_model.encoder.causal_mask = my_model.encoder._create_lookahead_mask(1500, 5, DEVICE, dtype=my_model.dtype)

In [27]:
my_model.encoder.causal_mask

tensor([[[[0., 0., 0.,  ..., -inf, -inf, -inf],
          [0., 0., 0.,  ..., -inf, -inf, -inf],
          [0., 0., 0.,  ..., -inf, -inf, -inf],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:6')

In [28]:
import inspect
print(inspect.getsource(my_model.__init__))

    def forward(self, input_features, **kwargs):
        # Run the standard preprocessing
        inputs_embeds = torch.nn.functional.gelu(self.conv1(input_features))
        inputs_embeds = torch.nn.functional.gelu(self.conv2(inputs_embeds))
        inputs_embeds = inputs_embeds.permute(0, 2, 1)
        
        # Positional encoding
        all_positions = torch.arange(self.embed_positions.num_embeddings, device=inputs_embeds.device)
        hidden_states = inputs_embeds + self.embed_positions(all_positions)
        hidden_states = torch.nn.functional.dropout(hidden_states, p=self.dropout, training=self.training)
        
        # Process through layers with mask
        encoder_states = () if kwargs.get('output_hidden_states', False) else None
        all_attentions = () if kwargs.get('output_attentions', False) else None
        
        for idx, encoder_layer in enumerate(self.layers):
            if kwargs.get('output_hidden_states', False):
                encoder_states = enco

In [21]:
my_model_loss = []
whisper_model_loss = []

for i in range(1):
    sample = ds[i]                                            # type: ignore[attr-defined]
    audio = torch.from_numpy(sample["mp3"]["array"]).float()  # type: ignore[attr-defined]
    text = sample["json"]["text"]                             # type: ignore[attr-defined]
    
    labels = torch.tensor(processor.tokenizer(text, add_special_tokens=False).input_ids).unsqueeze(0)  # type: ignore[attr-defined]
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt")  
    
    labels = labels.to(DEVICE)
    inputs = inputs.to(DEVICE)
    
    with torch.no_grad():
        my_latents = my_model.encoder(
            inputs.input_features 
            )
        
        my_outputs = my_model(
            inputs.input_features,
            labels=labels
        )
        
    # my_results = processor.batch_decode(my_outputs.logits.argmax(dim=-1), skip_special_tokens=True)[0]  # type: ignore[attr-defined]
    # whisper_results = processor.batch_decode(whisper_outputs.logits.argmax(dim=-1), skip_special_tokens=True)[0]  # type: ignore[attr-defined]
        
    # my_model_loss.append(my_outputs.loss.item())
    # whisper_model_loss.append(whisper_outputs.loss.item())
    # print("\n")
    
    print(my_latents.last_hidden_state.shape)
    my_results = processor.batch_decode(my_outputs.logits.argmax(dim=-1), skip_special_tokens=True)[0]  # type: ignore[attr-defined]

TypeError: WhisperModel.forward() got an unexpected keyword argument 'labels'